## ReMECS -- Real-time Multimodal Emotion Classification Syatem

## Using multimodal data stream [EEG+EDA+RESP_BELT] 

## Using LSTM or Bi-LSTM + SGD + One pass 

## Modifications:
- Reward Penalty based Weighted Ensemble [RPWE]


## Date: 23 Sept 2021 at 10:10 AM

In [2]:
#============================
# Import important libraries
#============================
import pandas as pd 
import numpy as np
import tensorflow as tf
import math
import scipy
import pywt
from creme import metrics
import time
import datetime
from statistics import mode
from scipy import stats
from sklearn import preprocessing
from collections import defaultdict, Counter
from scipy.special import expit
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

from window_slider import Slider
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dropout

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values)> 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):    
    list_values = list_values[0,:]
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

In [4]:
#======================================================
# EDA Feature Extraction (Wavelet Features)
#======================================================
def extract_eda_features(raw_eda):
    features =[]
    EDA = raw_eda
    list_coeff = pywt.wavedec(EDA, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features


In [5]:
#======================================================
# ECG Feature Extraction (Wavelet Features)
#======================================================

def extract_ecg_features(raw_data):
    ch =0
    features =[]
    
    def get_features(list_values):    
        list_values = list_values
        entropy = calculate_entropy(list_values)
        crossings = calculate_crossings(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + crossings + statistics

    for i in range(raw_data.shape[0]):
            ch_data = raw_data[i]
            list_coeff = pywt.wavedec(ch_data, 'db4', level=3)
            for coeff in list_coeff:
                features += get_features(coeff)
            ch = ch+1

    return features

In [6]:
def eeg_features(raw_data):
    ch = 0
    features= []
    def calculate_entropy(list_values):
        counter_values = Counter(list_values).most_common()
        probabilities = [elem[1]/len(list_values) for elem in counter_values]
        entropy=scipy.stats.entropy(probabilities)
        return entropy

    def calculate_statistics(list_values):
        median = np.nanpercentile(list_values, 50)
        mean = np.nanmean(list_values)
        std = np.nanstd(list_values)
        var = np.nanvar(list_values)
        rms = np.nanmean(np.sqrt(list_values**2))
        return [median, mean, std, var, rms]

    def get_features(list_values):    
    #     list_values = list_values[0,:]
        entropy = calculate_entropy(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + statistics
    
    for i in range(raw_data.shape[0]):
        ch_data = raw_data[i]
        list_coeff = pywt.wavedec(ch_data, 'db4', level=5)
        for coeff in list_coeff:
            features += get_features(coeff)
            
        ch = ch+1
    return features

In [7]:
# ##===================================================
# # EEG data read from files
# ##===================================================
# def eeg_data(p,v):
#     file_eeg = '/home/gp/Desktop/MER_arin/data/eeg_data/'+str(p)+'_data_DEAP'+'.csv'
#     print(file_eeg)
#     df = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
#     eeg_sig = df.loc[df.iloc[:,1] == v]
#     return eeg_sig

# ##===================================================
# # EDA data read from files
# ##===================================================
# def eda_data(p,v):
#     file_eda = '/home/gp/Desktop/MER_arin/data/eda_data/'+str(p)+'_GSR_data_from_DEAP.csv'
#     print(file_eda)
#     df = pd.read_csv(file_eda,sep=',', header = None, engine='python')
#     eda_sig = df.loc[df.iloc[:,1] == v]
#     return eda_sig

# ##===================================================
# # Resp data read from files
# ##===================================================
# def resp_data(p,v):
#     file_resp = '/home/gp/Desktop/MER_arin/data/resp_data/'+str(p)+'_Respiration_data_from_DEAP.csv'
#     print(file_resp)
#     df = pd.read_csv(file_resp,sep=',', header = None, engine='python')
#     resp_sig = df.loc[df.iloc[:,1] == v]
#     return resp_sig

In [8]:
# #============================
# # Bi-directional LSTM model 
# #============================
# def create_model(x):
#     model = Sequential() 
    
#     model.add(Bidirectional(LSTM(units = 30, return_sequences = True, input_shape = (1,x.shape[1]))))

#     model.add(Bidirectional(LSTM(units = 30, activation='sigmoid',return_sequences = True)))
    
#     model.add(Dense(1, activation='sigmoid'))
    
#     opt = tf.keras.optimizers.SGD(learning_rate=0.05)
    
#     model.compile(loss='binary_crossentropy',
#                   optimizer=opt,
#                   metrics=['accuracy'])
    
    
# #     print(model.summary())
    
#     return model

In [13]:
#==================================
# LSTM model
#==================================
def create_model(x):
    model = Sequential() 
    
    model.add(LSTM(units = 2, activation='sigmoid' ,return_sequences = True, input_shape = (1,x.shape[1])))

    model.add(LSTM(units = 2, activation='sigmoid',return_sequences = True))
    
    model.add(Dense(1, activation='sigmoid'))
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.05)
    
    model.compile(loss='mse',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    
    print(model.summary())
    
    return model

In [14]:
#================================================================
# Initialization of different parameters and performance metrics
#================================================================
segment_in_sec = 10 #in sec
bucket_size = int((7085/51)*segment_in_sec)  #7085 is for 51 sec video record
overlap_count = 0

num_classifiers = 3 #Total number of classifiers
w_val =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers
w_aro =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers

cBbest_val = [] #Classifiers least error for valence classsification
cBest_aro = []  #Classifiers least error for arousal classsification
cWorst_val = [] #Classifiers heighest error for valence classsification
cWorst_aro = []  #Classifiers heighest error for arousal classsification

beta_val = np.ones(num_classifiers)/num_classifiers
beta_aro = np.ones(num_classifiers)/num_classifiers


optimizer= 'SGD' #optimizer
classifier = 'LSTM_RPWE'+str(optimizer)
indx = 0
c=0


participant = 40
videos = 16

global eeg_emotion, eda_emotion, ecg_emotion, mer_emotion, all_eta
eeg_emotion = []
eda_emotion = []
ecg_emotion = []
mer_emotion = []

all_eta =[]
init_m = 0

#================================================
# Performance matric declaration here
#================================================
mer_acc_val = metrics.Accuracy() #Accuracy
mer_f1m_val = metrics.F1() #F1 measure  
mer_acc_aro = metrics.Accuracy() #Accuracy
mer_f1m_aro = metrics.F1() #F1 measure
mer_roc_val = metrics.ROCAUC()
mer_roc_aro = metrics.ROCAUC() 
mer_mcc_val = metrics.MCC()
mer_mcc_aro = metrics.MCC()


eeg_acc_val = metrics.Accuracy() #Accuracy
eeg_f1m_val = metrics.F1() #F1 measure  
eeg_acc_aro = metrics.Accuracy() #Accuracy
eeg_f1m_aro = metrics.F1() #F1 measure
eeg_roc_val = metrics.ROCAUC()
eeg_roc_aro = metrics.ROCAUC()
eeg_mcc_val = metrics.MCC()
eeg_mcc_aro = metrics.MCC()


eda_acc_val = metrics.Accuracy() #Accuracy
eda_f1m_val = metrics.F1() #F1 measure  
eda_acc_aro = metrics.Accuracy() #Accuracy
eda_f1m_aro = metrics.F1() #F1 measure
eda_roc_val = metrics.ROCAUC()
eda_roc_aro = metrics.ROCAUC()
eda_mcc_val = metrics.MCC()
eda_mcc_aro = metrics.MCC()


ecg_acc_val = metrics.Accuracy() #Accuracy
ecg_f1m_val = metrics.F1() #F1 measure  
ecg_acc_aro = metrics.Accuracy() #Accuracy
ecg_f1m_aro = metrics.F1() #F1 measure
ecg_roc_val = metrics.ROCAUC()
ecg_roc_aro = metrics.ROCAUC()
ecg_mcc_val = metrics.MCC()
ecg_mcc_aro = metrics.MCC()

val_MSE = []
aro_MSE = []

haha_val = []
haha_aro = []


 
mer_cm_val  = metrics.ConfusionMatrix()
mer_cm_aro  = metrics.ConfusionMatrix()

eeg_cm_val  = metrics.ConfusionMatrix()
eeg_cm_aro  = metrics.ConfusionMatrix()

eda_cm_val  = metrics.ConfusionMatrix()
eda_cm_aro  = metrics.ConfusionMatrix()
  
ecg_cm_val  = metrics.ConfusionMatrix()
ecg_cm_aro  = metrics.ConfusionMatrix()


    
itr = 0 #controls the learning rate


#=======================================
# MAIN Loop STARTS HERE
#=======================================
for ii in range(0,participant):

        p =ii+1 #participant number
        
        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')
        ##===================================================
        # EEG data read from files
        ##===================================================
        def eeg_data(p):
            file_eeg = '/home/gp/Desktop/MER_arin/AMIGOS_data/eeg_data/'+str(p)+'data_eeg_AMIGOS'+'.csv'
            print(file_eeg)
            eeg_sig = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
            return eeg_sig

        ##===================================================
        # EDA data read from files
        ##===================================================
        def eda_data(p):
            file_eda = '/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/'+str(p)+'data_eda_AMIGOS.csv'
            print(file_eda)
            eda_sig = pd.read_csv(file_eda,sep=',', header = None, engine='python')
            return eda_sig

        ##===================================================
        # ECG data read from files
        ##===================================================
        def ecg_data(p):
            file_ecg = '/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/'+str(p)+'data_ecg_AMIGOS.csv'
            print(file_ecg)
            ecg_sig = pd.read_csv(file_ecg,sep=',', header = None, engine='python')
            return ecg_sig
        
        #------------------------------------
        # Once file fetched data stored here 
        #------------------------------------
        grand_eeg = eeg_data(p)
        grand_eda = eda_data(p)
        grand_ecg = ecg_data(p)

        
        for jj in range(0,videos):
            v = jj+1 #Video number
            print('=========================================================================')
            p_v = 'Person:'+ ' ' +str(p)+ ' ' +'Video:'+str(v)
            print(p_v)
            
            emotion_label =[]
            
            
            ##===================================================
            # Data read from files
            ##===================================================
            eeg_sig = grand_eeg.loc[grand_eeg.iloc[:,1] == v]
            eda_sig = grand_eda.loc[grand_eda.iloc[:,1] == v]
            ecg_sig = grand_ecg.loc[grand_ecg.iloc[:,1] == v]
            
            #=================================================
            #emotion labels (valence, arousal) mapping 0-1
            #=================================================
            aro = eeg_sig.iloc[0,7087]
            val = eeg_sig.iloc[0,7088]
            
            #valence emotion maping 0-> low valence and 1-> high valence

            if (val >5):
                vl = 1 #high valence
            else:
                vl = 0 #low valence

            #arousal emotion maping 0-> low arousal and 1-> high high arousal
            if (aro >5):
                al = 1 #high arousal
            else:
                al = 0 #low arousal
                
            y_act_val = np.array([vl])
            y_act_aro = np.array([al]) 
            
            #==========================================================
            # Predicted Valence and Arousal labels list initialization
            #==========================================================
            eeg_val_prdt=[]
            eeg_aro_prdt =[]
            
            eda_val_prdt=[]
            eda_aro_prdt =[]
            
            ecg_val_prdt=[]
            ecg_aro_prdt =[]
            
            
            #=========================================
            # Sliding window starts here 
            #=========================================
            slider_eeg = Slider(bucket_size,overlap_count)
            slider_eda = Slider(bucket_size,overlap_count)
            slider_ecg = Slider(bucket_size,overlap_count)
            
            eeg_sig = np.array(eeg_sig.iloc[range(0,14),range(2,7087)]) #keeping only eeg signals
            eda_sig = np.array(eda_sig.iloc[:,range(2,7087)]) #keeping only eda signals
            ecg_sig = np.array(ecg_sig.iloc[range(0,2),range(2,7087)]) #keeping only ecg signals
            
            slider_eeg.fit(eeg_sig)
            slider_eda.fit(eda_sig)
            slider_ecg.fit(ecg_sig)

            while True:
                window_data_eeg = slider_eeg.slide()
                window_data_eda = slider_eda.slide() 
                window_data_ecg = slider_ecg.slide() 
               
                
                #=================================================
                # Feature extraction from EEG
                #=================================================
                window_data_eeg  = np.nan_to_num(window_data_eeg) #Replacing NaN with 0 if there is such case
                features_eeg = eeg_features(window_data_eeg)
                eeg = np.array([features_eeg])  #EEG raw feature vector
                x_eeg = preprocessing.normalize(eeg) # EEG normalized features [0,1] 
                
                
                #=================================================
                # Feature extraction from EDA
                #=================================================
                window_data_eda  = np.nan_to_num(window_data_eda) #Replacing NaN with 0 if there is such case
                eda_features = extract_eda_features(np.array(window_data_eda))
                eda = np.array([eda_features]) #EDA raw feature vector
                x_eda = preprocessing.normalize(eda) #EDA normalized features
                
                #=================================================
                # Feature extraction from ecg belt
                #=================================================
                window_data_ecg  = np.nan_to_num(window_data_ecg) #Replacing NaN with 0 if there is such case
                ecg_features = extract_ecg_features(np.array(window_data_ecg))
                ecg = np.array([ecg_features]) #ecg BELT raw feature vector
                x_ecg = preprocessing.normalize(ecg) #ecg BELT normalized features
            
            
                #===================================================
                # Model initialization
                #===================================================
                if init_m == 0:
                    print('EEG Feature shape{}:'.format(x_eeg.shape))
                    print('EDA Feature shape{}:'.format(x_eda.shape))
                    print('ecg BELT Feature shape{}:'.format(x_ecg.shape))

                    #========================
                    # For EEG data model
                    #========================
                    eeg_model_val = create_model(x_eeg)

                    eeg_model_aro = create_model(x_eeg)

                    #========================
                    # For EDA data model
                    #========================
                    eda_model_val = create_model(x_eda)

                    eda_model_aro = create_model(x_eda)

                    #==============================
                    # For ecg Belt data Model
                    #==============================
                    ecg_model_val = create_model(x_ecg)

                    ecg_model_aro = create_model(x_ecg)

                    init_m = init_m+1


                #===============================================================
                # Emotion Classification --> Valence and Arousal
                #===============================================================

                if c == 0: #For the first time model will return 0 or None
                    tmp_eeg_val = np.array([0])
                    tmp_eeg_aro = np.array([0])
                    
                    tmp_eda_val = np.array([0])
                    tmp_eda_aro = np.array([0])
                    
                    tmp_ecg_val = np.array([0])
                    tmp_ecg_aro = np.array([0])
                    
                    
                    y_pred_val_eeg = np.array([2])
                    y_pred_aro_eeg = np.array([2])
                    
                    y_pred_val_eda = np.array([2])
                    y_pred_aro_eda = np.array([2])
                    
                    y_pred_val_ecg = np.array([2])
                    y_pred_aro_ecg = np.array([2])
                    
                    eeg_model_val.fit(np.array([x_eeg]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eeg_model_aro.fit(np.array([x_eeg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    eda_model_val.fit(np.array([x_eda]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eda_model_aro.fit(np.array([x_eda]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    ecg_model_val.fit(np.array([x_ecg]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    ecg_model_aro.fit(np.array([x_ecg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    c=c+1
                    
                else: 
                    
                    tmp_eeg_val  = eeg_model_val.predict(np.array([x_eeg]))
                    tmp_eeg_aro = eeg_model_aro.predict(np.array([x_eeg]))
                    
                    tmp_eda_val = eda_model_val.predict(np.array([x_eda]))
                    tmp_eda_aro  = eda_model_aro.predict(np.array([x_eda]))
                    
                    tmp_ecg_val = ecg_model_val.predict(np.array([x_ecg]))
                    tmp_ecg_aro = ecg_model_aro.predict(np.array([x_ecg]))
                    
                    
                    #Binary output scaling 1 if x > 0.5 else 0
                    y_pred_val_eeg = [1 if tmp_eeg_val > 0.5 else 0]
                    y_pred_aro_eeg = [1 if tmp_eeg_aro > 0.5 else 0]
                    
                    y_pred_val_eda = [1 if tmp_eda_val > 0.5 else 0]
                    y_pred_aro_eda = [1 if tmp_eda_aro > 0.5 else 0]
                    
                    y_pred_val_ecg = [1 if tmp_ecg_val > 0.5 else 0]
                    y_pred_aro_ecg = [1 if tmp_ecg_aro > 0.5 else 0]
                    
                    #Fit the model once
                    eeg_model_val.fit(np.array([x_eeg]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eeg_model_aro.fit(np.array([x_eeg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    eda_model_val.fit(np.array([x_eda]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eda_model_aro.fit(np.array([x_eda]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    ecg_model_val.fit(np.array([x_ecg]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    ecg_model_aro.fit(np.array([x_ecg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    

                #===========================================
                # Performance matric update
                #===========================================
                
                #Valence classification EEG

                eeg_acc_val = eeg_acc_val.update(y_act_val[0], y_pred_val_eeg[0])  # update the accuracy metric

                eeg_f1m_val = eeg_f1m_val.update(y_act_val[0], y_pred_val_eeg[0]) #update f1 measure metric
                
                eeg_roc_val = eeg_roc_val.update(y_act_val[0], y_pred_val_eeg[0])
                
                eeg_mcc_val = eeg_mcc_val.update(y_act_val[0], y_pred_val_eeg[0])
                
                #Arousal classification EEG 

                eeg_acc_aro = eeg_acc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])  # update the accuracy metric

                eeg_f1m_aro = eeg_f1m_aro.update(y_act_aro[0], y_pred_aro_eeg[0]) #update f1 measure metric
                
                eeg_roc_aro = eeg_roc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])
                
                eeg_mcc_aro = eeg_mcc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])


                #Valence classification EDA

                eda_acc_val = eda_acc_val.update(y_act_val[0], y_pred_val_eda[0])  # update the accuracy metric

                eda_f1m_val = eda_f1m_val.update(y_act_val[0], y_pred_val_eda[0]) #update f1 measure metric
                
                eda_roc_val = eda_roc_val.update(y_act_val[0], y_pred_val_eda[0])
                
                eda_mcc_val = eda_mcc_val.update(y_act_val[0], y_pred_val_eda[0])
                

                #Arousal classification EDA
                
                eda_acc_aro = eda_acc_aro.update(y_act_aro[0], y_pred_aro_eda[0])  # update the accuracy metric

                eda_f1m_aro = eda_f1m_aro.update(y_act_aro[0], y_pred_aro_eda[0]) #update f1 measure metric
                
                eda_roc_aro = eda_roc_aro.update(y_act_aro[0], y_pred_aro_eda[0])
                
                eda_mcc_aro = eda_mcc_aro.update(y_act_aro[0], y_pred_aro_eda[0])

                #Valence classification ecg Belt

                ecg_acc_val = ecg_acc_val.update(y_act_val[0], y_pred_val_ecg[0])  # update the accuracy metric

                ecg_f1m_val = ecg_f1m_val.update(y_act_val[0], y_pred_val_ecg[0]) #update f1 measure metric
                
                ecg_roc_val = ecg_roc_val.update(y_act_val[0], y_pred_val_ecg[0])
                
                ecg_mcc_val = ecg_roc_val.update(y_act_val[0], y_pred_val_ecg[0])
                

                #Arousal classification ecg Belt

                ecg_acc_aro = ecg_acc_aro.update(y_act_aro[0], y_pred_aro_ecg[0])  # update the accuracy metric

                ecg_f1m_aro = ecg_f1m_aro.update(y_act_aro[0], y_pred_aro_ecg[0]) #update f1 measure metric
                
                ecg_roc_aro = ecg_roc_aro.update(y_act_aro[0], y_pred_aro_ecg[0])
                
                ecg_mcc_aro = ecg_mcc_aro.update(y_act_aro[0], y_pred_aro_ecg[0])
                

                if slider_eeg.reached_end_of_list(): break
                
            #===================================================
            #   Error storing and comparing Part of each models
            #===================================================
            
            eeg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eeg_val))[0][0][0]
            eeg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eeg_aro))[0][0][0]
            
            eda_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eda_val))[0][0][0]
            eda_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eda_aro))[0][0][0]
            
            ecg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_ecg_val))[0][0][0]
            ecg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_ecg_aro))[0][0][0]
            
            val_MSE = [eeg_val_MSE, eda_val_MSE, ecg_val_MSE] #Valence errors 
            aro_MSE = [eeg_aro_MSE, eda_aro_MSE, ecg_aro_MSE] #Arousal errors
            

            
            #========================================================
            # Valence Arousal MSE Compariosn and storing in Cbest and Cworst
            #========================================================
            
            if (jj==0): #First Video
                cBest_val = val_MSE
                cBest_aro = aro_MSE
                
                cWorst_val  = val_MSE
                cWorst_aro = aro_MSE
                
            else:
                #-----------------------------------------
                #Beta for calence and arousal calculation
                #-----------------------------------------              
                beta_val = np.true_divide(list(np.subtract(cBest_val,val_MSE)), (1+np.exp(list(np.subtract(cWorst_val,cBest_val)))))
                beta_aro = np.true_divide(list(np.subtract(cBest_aro,aro_MSE)), (1+np.exp(list(np.subtract(cWorst_aro,cBest_aro)))))
                
                #--------------------------------------------------------------
                # Classifiers best MSE error and worst MSE error value update
                #-------------------------------------------------------------
                cBest_val = np.minimum(cBest_val, val_MSE)
                cBest_aro = np.minimum(cBest_aro, aro_MSE)
                cWorst_val = np.maximum(cWorst_val, val_MSE)
                cWorst_aro = np.maximum(cWorst_aro, aro_MSE)
            ##=============================
            # Confusion Matric Calculation
            ##=============================
            
            eeg_cm_val = eeg_cm_val.update(y_act_val[0], y_pred_val_eeg[0])
            eeg_cm_aro = eeg_cm_aro.update(y_act_aro[0], y_pred_aro_eeg[0])
            
            eda_cm_val = eda_cm_val.update(y_act_val[0], y_pred_val_eda[0])
            eda_cm_aro = eda_cm_aro.update(y_act_aro[0], y_pred_aro_eda[0])
            
            ecg_cm_val = ecg_cm_val.update(y_act_val[0], y_pred_val_ecg[0])
            ecg_cm_varo = ecg_cm_aro.update(y_act_aro[0], y_pred_aro_ecg[0])            
            
            #=============================================================
            # Storing All Results
            #=============================================================
            
            emotion_label.append([y_pred_val_eeg[0],y_pred_aro_eeg[0]]) #appending valence & arousal predicted EEG
            emotion_label.append([y_pred_val_eda[0], y_pred_aro_eda[0]]) #appending valence & arousal predicted EDA
            emotion_label.append([y_pred_val_ecg[0], y_pred_aro_ecg[0]]) #appending valence & arousal predicted ecg

            emotion_label = np.array(emotion_label)
            
                    
            #====================================================================
            # Decision label ensemble --> Reward Penalty Based Weightrd Ensemble
            #====================================================================
            val_label = emotion_label[:,0]
            aro_label = emotion_label[:,1]
            
            #------------------------------------------
            # Valence Class ensemble
            #------------------------------------------
            
            p_val = np.dot([w_val],val_label)
            y_prdt_mer_val = (p_val -0.5)
            if(y_prdt_mer_val > 0):
                mer_val = 1
            else:
                mer_val = 0

            #============================================
            # Weight update for Valence
            #============================================
            w_val = w_val + list(np.multiply(w_val,beta_val))    
            w_val_sum = sum(w_val) #total sum of weights
            w_val = np.array(w_val/w_val_sum) #weight rescaling


            
            #------------------------------------------
            # Arousal Class ensemble
            #------------------------------------------            
            p_val = np.dot([w_aro],aro_label)
            y_prdt_mer_aro = (p_val-0.5)
            if(y_prdt_mer_aro > 0):
                mer_aro = 1
            else:
                mer_aro = 0

            w_aro = w_aro + list(np.multiply(w_aro,beta_aro))
            w_aro_sum = sum(w_aro) #total sum of weights
            w_aro = np.array(w_aro/w_aro_sum) #weight rescaling
            
            #========================================================
            # ReMECS performance metric 
            #========================================================
            mer_acc_val = mer_acc_val.update(y_act_val[0], mer_val)
            mer_f1m_val = mer_f1m_val.update(y_act_val[0], mer_val)
            mer_roc_val = mer_roc_val.update(y_act_val[0], mer_val)
            mer_mcc_val = mer_mcc_val.update(y_act_val[0], mer_val)
            mer_cm_val  = mer_cm_val.update(y_act_val[0], mer_val)
            
            mer_acc_aro = mer_acc_aro.update(y_act_aro[0], mer_aro)
            mer_f1m_aro = mer_f1m_aro.update(y_act_aro[0], mer_aro)
            mer_roc_aro = mer_roc_aro.update(y_act_aro[0], mer_aro)
            mer_mcc_aro = mer_mcc_aro.update(y_act_aro[0], mer_aro)
            mer_cm_aro  = mer_cm_aro.update(y_act_aro[0], mer_aro) 
            
            
            eeg_emotion.append(np.array([p,v,eeg_acc_val.get(), eeg_f1m_val.get(), eeg_roc_val.get(),eeg_mcc_val.get(),
                                         eeg_acc_aro.get(), eeg_f1m_aro.get(),eeg_roc_aro.get(), eeg_mcc_aro.get(), 
                                         y_act_val[0], y_pred_val_eeg[0], y_act_aro[0], y_pred_aro_eeg[0]]))
            
            eda_emotion.append(np.array([p,v,eda_acc_val.get(), eda_f1m_val.get(), eda_roc_val.get(),eda_mcc_val.get(),
                                         eda_acc_aro.get(), eda_f1m_aro.get(), eda_roc_aro.get(), eda_mcc_aro.get(),
                                         y_act_val[0], y_pred_val_eda[0], y_act_aro[0], y_pred_aro_eda[0]]))
            
            ecg_emotion.append(np.array([p,v, ecg_acc_val.get(), ecg_f1m_val.get(), ecg_roc_val.get(), ecg_mcc_val.get(),
                                          ecg_acc_aro.get(),ecg_f1m_aro.get(), ecg_roc_aro.get(), ecg_mcc_aro.get(), 
                                          y_act_val[0], y_pred_val_ecg[0], y_act_aro[0], y_pred_aro_ecg[0]]))
            
            mer_emotion.append(np.array([p,v,mer_acc_val.get(), mer_f1m_val.get(), mer_roc_val.get(), mer_mcc_val.get(),
                                         mer_acc_aro.get(), mer_f1m_aro.get(), mer_roc_aro.get(), mer_mcc_aro.get(), 
                                         y_act_val[0], mer_val, y_act_aro[0], mer_aro]))
            print('Actual emotion [valence: {}, arousal: {}]'.format(y_act_val[0], y_act_aro[0]))
            print('Single modals Predicted emotion')
            print('[valence, arousal]:')
            print(emotion_label)
            
            print('MER Predicted emotion [valence: {}, arousal: {}]'.format(mer_val, mer_aro))
#             print('Val MSE',val_MSE)
#             print('Aro MSE',aro_MSE)
#             # print('------------------------------')
#             print('Beta_Val:', beta_val)
#             print('Beta_Aro:', beta_aro)
#             print('------------------------------')
#             print('W Val:', w_val)
#             print('W Aro:', w_aro)
            # print('------------------------------')
            # print('CBest Val MSE',cBest_val)
            # print('CBest Aro MSE',cBest_aro)
            # print('------------------------------')
            # print('Cworst Val MSE',cWorst_val)
            # print('Cworst Aro MSE',cWorst_aro)
            # print('==============================================')            
            # print('..................................................')
            # print('EEG')

            # print("Valence Accuracy:{}".format(round(eeg_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eeg_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eeg_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eeg_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eeg_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eeg_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eeg_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eeg_mcc_aro.get(),4)))

            # print('..................................................')
            # print('EDA')

            # print("Valence Accuracy:{}".format(round(eda_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eda_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eda_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eda_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eda_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eda_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eda_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eda_mcc_aro.get(),4)))


            # print('..................................................')
            # print('ecg BELT')

            # print("Valence Accuracy:{}".format(round(ecg_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(ecg_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(ecg_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(ecg_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(ecg_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(ecg_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(ecg_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(ecg_mcc_aro.get(),4)))

            # print('..................................................')
            print('MER model')

            print("Valence Accuracy:{}".format(round(mer_acc_val.get(),4)))
            print("Valence F1 score:{}".format(round(mer_f1m_val.get(),4)))
            print("Valence ROC score:{}".format(round(mer_roc_val.get(),4)))
            print("Valence MCC score:{}".format(round(mer_mcc_val.get(),4)))
            
            print("Arousal Accuracy:{}".format(round(mer_acc_aro.get(),4)))
            print("Arousal F1 score:{}".format(round(mer_f1m_aro.get(),4)))
            print("Arousal ROC score:{}".format(round(mer_roc_aro.get(),4)))
            print("Arousal MCC score:{}".format(round(mer_mcc_aro.get(),4)))
            print('===============================xxxx================================')
            



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/MER_arin/AMIGOS_data/eeg_data/1data_eeg_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/1data_eda_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/1data_ecg_AMIGOS.csv
Person: 1 Video:1
EEG Feature shape(1, 504):
EDA Feature shape(1, 48):
ecg BELT Feature shape(1, 96):
Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_36 (LSTM)               (None, 1, 2)              4056      
_________________________________________________________________
lstm_37 (LSTM)               (None, 1, 2)              40        
_________________________________________________________________
dense_18 (Dense)             (None, 1, 1)              3         
Total params: 4,099
Trainable params: 4,099
Non-trainable params: 0
_____________________________________

Actual emotion [valence: 1, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [0 0]
 [1 0]]
MER Predicted emotion [valence: 1, arousal: 0]
MER model
Valence Accuracy:0.5
Valence F1 score:0.6667
Valence ROC score:0.5
Valence MCC score:0.0
Arousal Accuracy:1.0
Arousal F1 score:0.0
Arousal ROC score:-0.0
Arousal MCC score:0.0
===============================xxxx================================
Person: 1 Video:5
Actual emotion [valence: 0, arousal: 1]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [0 0]
 [1 0]]
MER Predicted emotion [valence: 1, arousal: 0]
MER model
Valence Accuracy:0.4
Valence F1 score:0.5714
Valence ROC score:0.5
Valence MCC score:0.0
Arousal Accuracy:0.8
Arousal F1 score:0.0
Arousal ROC score:0.5
Arousal MCC score:0.0
===============================xxxx================================
Person: 1 Video:6
Actual emotion [valence: 0, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [0 0]
 [0 0]]
MER Predicted emotion

Actual emotion [valence: 1, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [1 0]
 [1 0]]
MER Predicted emotion [valence: 1, arousal: 0]
MER model
Valence Accuracy:0.65
Valence F1 score:0.6957
Valence ROC score:0.6414
Valence MCC score:0.2872
Arousal Accuracy:0.65
Arousal F1 score:0.0
Arousal ROC score:0.5
Arousal MCC score:0.0
===============================xxxx================================
Person: 2 Video:5
Actual emotion [valence: 0, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [1 0]
 [1 0]]
MER Predicted emotion [valence: 1, arousal: 0]
MER model
Valence Accuracy:0.619
Valence F1 score:0.6667
Valence ROC score:0.6136
Valence MCC score:0.2337
Arousal Accuracy:0.6667
Arousal F1 score:0.0
Arousal ROC score:0.5
Arousal MCC score:0.0
===============================xxxx================================
Person: 2 Video:6
Actual emotion [valence: 0, arousal: 1]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [1 0]
 [1 0]]
MER

Actual emotion [valence: 1, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [1 0]
 [1 0]]
MER Predicted emotion [valence: 1, arousal: 0]
MER model
Valence Accuracy:0.5833
Valence F1 score:0.5946
Valence ROC score:0.5833
Valence MCC score:0.1669
Arousal Accuracy:0.6944
Arousal F1 score:0.0
Arousal ROC score:0.5
Arousal MCC score:0.0
===============================xxxx================================
Person: 3 Video:5
Actual emotion [valence: 0, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [1 0]
 [1 0]]
MER Predicted emotion [valence: 1, arousal: 0]
MER model
Valence Accuracy:0.5676
Valence F1 score:0.5789
Valence ROC score:0.5687
Valence MCC score:0.1378
Arousal Accuracy:0.7027
Arousal F1 score:0.0
Arousal ROC score:0.5
Arousal MCC score:0.0
===============================xxxx================================
Person: 3 Video:6
Actual emotion [valence: 0, arousal: 1]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [0 0]
 [0 0]

Actual emotion [valence: 1, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5769
Valence F1 score:0.5
Valence ROC score:0.5685
Valence MCC score:0.1403
Arousal Accuracy:0.6538
Arousal F1 score:0.0
Arousal ROC score:0.5
Arousal MCC score:0.0
===============================xxxx================================
Person: 4 Video:5
Actual emotion [valence: 0, arousal: 1]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5849
Valence F1 score:0.5
Valence ROC score:0.574
Valence MCC score:0.152
Arousal Accuracy:0.6415
Arousal F1 score:0.0
Arousal ROC score:0.5
Arousal MCC score:0.0
===============================xxxx================================
Person: 4 Video:6
Actual emotion [valence: 0, arousal: 1]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Pr

...

/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/36data_eda_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/36data_ecg_AMIGOS.csv
Person: 36 Video:1
Actual emotion [valence: 1, arousal: 1]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5954
Valence F1 score:0.5487
Valence ROC score:0.593
Valence MCC score:0.1891
Arousal Accuracy:0.6435
Arousal F1 score:0.6774
Arousal ROC score:0.6393
Arousal MCC score:0.2807
===============================xxxx================================
Person: 36 Video:2
Actual emotion [valence: 1, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5943
Valence F1 score:0.5476
Valence ROC score:0.5921
Valence MCC score:0.1873
Arousal Accuracy:0.6441
Arousal F1 score:0.6774
Arousal ROC score:0.6401
Arousal MCC score:0.2822
=======================

/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/37data_eda_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/37data_ecg_AMIGOS.csv
Person: 37 Video:1
Actual emotion [valence: 1, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5962
Valence F1 score:0.5422
Valence ROC score:0.5929
Valence MCC score:0.1897
Arousal Accuracy:0.6447
Arousal F1 score:0.672
Arousal ROC score:0.6421
Arousal MCC score:0.2854
===============================xxxx================================
Person: 37 Video:2
Actual emotion [valence: 1, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5952
Valence F1 score:0.5412
Valence ROC score:0.5921
Valence MCC score:0.188
Arousal Accuracy:0.6453
Arousal F1 score:0.672
Arousal ROC score:0.6428
Arousal MCC score:0.2868
=========================

/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/38data_eda_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/38data_ecg_AMIGOS.csv
Person: 38 Video:1
Actual emotion [valence: 0, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.597
Valence F1 score:0.5359
Valence ROC score:0.5928
Valence MCC score:0.1902
Arousal Accuracy:0.6459
Arousal F1 score:0.6667
Arousal ROC score:0.6444
Arousal MCC score:0.2893
===============================xxxx================================
Person: 38 Video:2
Actual emotion [valence: 1, arousal: 1]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.596
Valence F1 score:0.5349
Valence ROC score:0.5919
Valence MCC score:0.1886
Arousal Accuracy:0.6448
Arousal F1 score:0.6656
Arousal ROC score:0.6433
Arousal MCC score:0.2871
========================

/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/39data_eda_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/39data_ecg_AMIGOS.csv
Person: 39 Video:1
Actual emotion [valence: 1, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5944
Valence F1 score:0.5277
Valence ROC score:0.5899
Valence MCC score:0.1854
Arousal Accuracy:0.642
Arousal F1 score:0.6594
Arousal ROC score:0.6411
Arousal MCC score:0.2822
===============================xxxx================================
Person: 39 Video:2
Actual emotion [valence: 1, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5934
Valence F1 score:0.5267
Valence ROC score:0.5891
Valence MCC score:0.1838
Arousal Accuracy:0.6426
Arousal F1 score:0.6594
Arousal ROC score:0.6417
Arousal MCC score:0.2835
=======================

/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/40data_eda_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/40data_ecg_AMIGOS.csv
Person: 40 Video:1
Actual emotion [valence: 0, arousal: 0]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5936
Valence F1 score:0.5208
Valence ROC score:0.5885
Valence MCC score:0.1833
Arousal Accuracy:0.6416
Arousal F1 score:0.6564
Arousal ROC score:0.641
Arousal MCC score:0.2819
===============================xxxx================================
Person: 40 Video:2
Actual emotion [valence: 1, arousal: 1]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5927
Valence F1 score:0.5198
Valence ROC score:0.5877
Valence MCC score:0.1818
Arousal Accuracy:0.6422
Arousal F1 score:0.6575
Arousal ROC score:0.6415
Arousal MCC score:0.2829
=======================

In [15]:
print('EEG Val CM:')
print(eeg_cm_val)

print('EEG aro CM:')
print(eeg_cm_aro)

print('EDA Val CM:')
print(eda_cm_val)

print('EDA aro CM:')
print(eda_cm_aro)

print('ecg Val CM:')
print(ecg_cm_val)
print('ecg Aro CM:')
print(ecg_cm_aro)

print('ReMECS Val CM:')
print(mer_cm_val)

print('ReMECS aro CM:')
print(mer_cm_aro)

EEG Val CM:
             0      1
      0    234     98
      1    166    142
EEG aro CM:
             0      1
      0    182    121
      1    110    227
EDA Val CM:
             0      1
      0    245     87
      1    173    135
EDA aro CM:
             0      1
      0    190    113
      1    116    221
ecg Val CM:
             0      1
      0    240     92
      1    169    139
ecg Aro CM:
             0      1
      0    190    113
      1    116    221
ReMECS Val CM:
             0      1
      0    240     92
      1    169    139
ReMECS aro CM:
             0      1
      0    191    112
      1    116    221


In [16]:
print('[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]')
#=============================
# EEG
#=============================
t_eeg_emotion =pd.DataFrame(eeg_emotion) 
t_eeg_emotion = t_eeg_emotion.iloc[:,range(2,10)]
t_eeg_emotion_avg = np.array(np.mean(t_eeg_emotion))
t_eeg_emotion_std = np.array(np.std(t_eeg_emotion))

print('EEG result Average:', t_eeg_emotion_avg)
print('EEG result std:', t_eeg_emotion_std)

#=============================
# EDA
#=============================
t_eda_emotion =pd.DataFrame(eda_emotion) 
t_eda_emotion = t_eda_emotion.iloc[:,range(2,10)]
t_eda_emotion_avg = np.array(np.mean(t_eda_emotion))
t_eda_emotion_std = np.array(np.std(t_eda_emotion))

print('EDA result Average:', t_eda_emotion_avg)
print('EDA result std:', t_eda_emotion_std)
#=============================
# ecg BELT
#=============================
t_ecg_emotion =pd.DataFrame(ecg_emotion) 
t_ecg_emotion = t_ecg_emotion.iloc[:,range(2,10)]
t_ecg_emotion_avg = np.array(np.mean(t_ecg_emotion))
t_ecg_emotion_std = np.array(np.std(t_ecg_emotion))

print('ecg Belt result Average:', t_ecg_emotion_avg)
print('ecg Belt result std:', t_ecg_emotion_std)
#=============================
# MER
#=============================
t_mer_emotion =pd.DataFrame(mer_emotion) 
t_mer_emotion = t_mer_emotion.iloc[:,range(2,10)]
t_mer_emotion_avg = np.array(np.mean(t_mer_emotion))
t_mer_emotion_std = np.array(np.std(t_mer_emotion))

print('MER result Average:', t_mer_emotion_avg)
print('MER result std:', t_mer_emotion_std)

[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]
EEG result Average: [0.53365632 0.51032749 0.53416327 0.07355756 0.60602772 0.62570175
 0.60290786 0.21760908]
EEG result std: [0.03584658 0.04317567 0.02729925 0.03188185 0.06776142 0.08905994
 0.05790268 0.06588172]
EDA result Average: [0.55117593 0.44898605 0.54107869 0.09132733 0.63321024 0.57527932
 0.61364519 0.23578033]
EDA result std: [0.02004896 0.09605414 0.0268201  0.02957881 0.03531463 0.19065951
 0.06845116 0.10988013]
ecg Belt result Average: [0.54219869 0.49087599 0.5392513  0.5392513  0.6358618  0.57750015
 0.6161078  0.24552471]
ecg Belt result std: [0.02799587 0.04579584 0.026212   0.026212   0.03544216 0.19124738
 0.06799178 0.09654096]
MER result Average: [0.59351262 0.54510544 0.5910151  0.18725208 0.65408585 0.59475489
 0.63401882 0.27878459]
MER result std: [0.02989464 0.0483272  0.02876731 0.03321069 0.0350127  0.19539127
 0.06891433 0.09764734]


In [17]:
time_stamp = datetime.datetime.now()
run =1
algo= 'LSTM/' 
filename = "/home/gp/Desktop/MER_arin/mer_codes/11AMIGOS_RPWE/Results_RPWE_AMIGOS/"+algo
fname_eeg = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EEG'+'_' +classifier+'_results.csv'
fname_eda = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EDA'+'_' +classifier+'_results.csv'
fname_ecg = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_ecg_BELT'+'_' +classifier+'_results.csv'
fname_mer = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_MER'+'_' +classifier+'_results.csv'

column_names = ['Person', 'Video', 'Acc_val', 'F1_val','roc_val', 'mcc_val','Acc_aro',
                'F1_aro', 'roc_aro', 'mcc_aro', 'y_act_val', 'y_pred_val', 'y_act_aro', 'y_pred_aro']

eeg_emotion = pd.DataFrame(eeg_emotion,columns = column_names)
eda_emotion = pd.DataFrame(eda_emotion,columns = column_names)
ecg_emotion = pd.DataFrame(ecg_emotion,columns = column_names)
mer_emotion = pd.DataFrame(mer_emotion,columns = column_names)

eeg_emotion.to_csv(fname_eeg)
eda_emotion.to_csv(fname_eda)
ecg_emotion.to_csv(fname_ecg)
mer_emotion.to_csv(fname_mer)
